In [ ]:
# Install packages needed for imports below
# ! pip3 install wavio 
# ! pip3 install librosa
# ! pip3 install sounddevice
# had to install an earlier version of matplotlib in order to get the librosa waveshow function to work
# ! pip3 install matplotlib==3.7.3

In [10]:
# import necessary modules
import numpy as np
from scipy.io import wavfile
from matplotlib import pyplot as plt
from scipy.fft import fft, fftfreq
from scipy.fft import rfft, rfftfreq
from scipy.io.wavfile import write
import matplotlib.pyplot as plt
import librosa.display
import IPython.display as ipd
import numpy as np
import time 

In [ ]:
# Define Sampling Rate or Frequency in Hz
sr = 44100

# Recording duration in seconds
duration = 5

def generate_sine_wave(freq, sample_rate, duration):
    x = np.linspace(0, duration, sample_rate*duration, endpoint=False)
    frequencies = x*freq
    y = np.sin((2*np.pi)*frequencies)
    return x, y

# generates a spectrogram from the recorded audio array
def generate_spectrogram(array, sr_in, plot_title):
    freq = librosa.amplitude_to_db(np.abs(librosa.stft(array)), ref=np.max)
    fig, ax = plt.subplots()
    plt.ylim(0,8000) # limit frequencies plotted to between 0 and 8000 Hz
    plt.title(plot_title + " Spectrogram") # insert plot title based on function input
    img = librosa.display.specshow(freq, x_axis='time', y_axis='linear',ax=ax, sr=sr_in)
    plt.xlabel("Time (seconds)")
    plt.ylabel("Frequency (Hertz)")
    fig.colorbar(img, ax=ax)

x1, tone1 = generate_sine_wave(5000, sr, duration)

write("team[]-sinetone.wav", sr, tone1)
generate_spectrogram(tone1, sr, "5000Hz Sine Wave")

In [ ]:
# Define Sampling Rate or Frequency in Hz
sr = 44100

# Recording duration in seconds
duration = 5

def generate_sine_wave(freq, sample_rate, duration):
    x = np.linspace(0, duration, sample_rate*duration, endpoint=False)
    frequencies = x*freq
    y = np.sin((2*np.pi)*frequencies)
    return x, y

# generates a spectrogram from the recorded audio array
def generate_spectrogram(array, sr_in, plot_title):
    freq = librosa.amplitude_to_db(np.abs(librosa.stft(array)), ref=np.max)
    fig, ax = plt.subplots()
    plt.ylim(0,8000) # limit frequencies plotted to between 0 and 8000 Hz
    plt.title(plot_title + " Spectrogram") # insert plot title based on function input
    img = librosa.display.specshow(freq, x_axis='time', y_axis='linear',ax=ax, sr=sr_in)
    plt.xlabel("Time (seconds)")
    plt.ylabel("Frequency (Hertz)")
    fig.colorbar(img, ax=ax)

x1, tone1 = generate_sine_wave(5000, sr, duration)

write("team[]-sinetone.wav", sr, tone1)
generate_spectrogram(tone1, sr, "5000Hz Sine Wave")